In [ ]:
import os
import requests
from openpyxl import load_workbook

# -----------------------------------------------------
# CONFIGURATION
# -----------------------------------------------------

xlsm_file = r"C:\Users\GWX-DATA\Documents\Rock 2\Nov25\24-30NOV.xlsm"       # <-- Update this
output_folder = r"C:\Users\GWX-DATA\Documents\Rock 2\Nov25\Images"   # <-- Update this
sheet_name = "24-30NOV"

# Create folder structure
folders = {
    "adhoc_landmark": os.path.join(output_folder, "adhoc", "landmark"),
    "adhoc_address":  os.path.join(output_folder, "adhoc", "address"),
    "rider_landmark": os.path.join(output_folder, "rider", "landmark"),
    "rider_address":  os.path.join(output_folder, "rider", "address"),
}

for f in folders.values():
    os.makedirs(f, exist_ok=True)

# Counters
counters = {
    "adhoc_landmark": 1,
    "adhoc_address": 1,
    "rider_landmark": 1,
    "rider_address": 1
}

# -----------------------------------------------------
# LOAD WORKBOOK
# -----------------------------------------------------
wb = load_workbook(xlsm_file, data_only=True)
ws = wb[sheet_name]

# -----------------------------------------------------
# DOWNLOAD IMAGES
# -----------------------------------------------------
for row in range(2, ws.max_row + 1):   # skip header row

    vocat = ws[f"M{row}"].value
    if vocat is None:
        continue

    vocat = str(vocat).strip().lower()

    # Read URLs
    landmark_url = ws[f"Q{row}"].value
    address_url  = ws[f"R{row}"].value

    # Select folder key
    if "adhoc" in vocat:
        base = "adhoc"
    elif "rider" in vocat:
        base = "rider"
    else:
        continue

    # DOWNLOAD LANDMARK IMAGE
    if landmark_url and isinstance(landmark_url, str) and landmark_url.startswith("http"):
        try:
            img = requests.get(landmark_url, timeout=10)
            if img.status_code == 200:
                key = f"{base}_landmark"
                file_path = os.path.join(folders[key], f"{key}_{counters[key]}.jpg")
                with open(file_path, "wb") as f:
                    f.write(img.content)
                counters[key] += 1
        except:
            pass

    # DOWNLOAD ADDRESS IMAGE
    if address_url and isinstance(address_url, str) and address_url.startswith("http"):
        try:
            img = requests.get(address_url, timeout=10)
            if img.status_code == 200:
                key = f"{base}_address"
                file_path = os.path.join(folders[key], f"{key}_{counters[key]}.jpg")
                with open(file_path, "wb") as f:
                    f.write(img.content)
                counters[key] += 1
        except:
            pass

print("✅ Finished downloading all images from Column Q & R.")
